In [ ]:
import requests
import numpy as np
import sklearn as sk
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy.random import rand

API_KEY = "pub_4dbeb1c8439e481cb04f5e6f2aaafe55"

url = "https://newsdata.io/api/1/news"

params = {
    "apikey": API_KEY,
    "q": "Ukraine",
    "language": "en"
}

response = requests.get(url, params=params)

data = response.json()

print(data.keys())

dict_keys(['status', 'totalResults', 'results', 'nextPage'])


In [25]:
articles = data['results']
# print(articles)
new_data = []
for article in articles:
  new_data.append(article.get('title') + " " + article.get('description'))

In [26]:
vectorizer = TfidfVectorizer(stop_words='english')
vec_data = vectorizer.fit_transform(new_data)

In [29]:
cov_matrix = B = vec_data.T @ vec_data

def find_q_vector(cov_matrix, k):
  rand_vector = np.random.rand(cov_matrix.shape[1])
  rand_vector = rand_vector / np.linalg.norm(rand_vector)

  for i in range(k):
    rand_vector = B @ rand_vector
    rand_vector = rand_vector / np.linalg.norm(rand_vector)
  return rand_vector

def reorthogonalize(vector_to_orth, list_of_vectors):
  for vector in list_of_vectors:
    #Reorthogonalization:
    vector_to_orth  = vector_to_orth - (vector.T @ vector_to_orth) * vector

    #Normalization:
    vector_to_orth = vector_to_orth / np.linalg.norm(vector_to_orth)
  return vector_to_orth

def find_eigenvector(first_vector, cov_matrix, k, n):
  # first_vector = find_q_vector(cov_matrix, k)
  list_of_eigenvectors = []

  lambda_vector = find_q_vector(cov_matrix, k)

  for i in range(n):
    lambda_vector = reorthogonalize(lambda_vector, list_of_eigenvectors)
    list_of_eigenvectors.append(lambda_vector)
